In [0]:
# Code from the next sources were used
#[4] A transformer chatbot tutorial with Tensorflow 2.0. Retrieved from https://medium.com/tensorflow/a-transformer-chatbot-tutorial-with-tensorflow-2-0-88bf59e66fe2
#[6] https://www.kaggle.com/maxwell110/beginner-s-guide-to-audio-data-2
#[7] https://www.kaggle.com/daisukelab/cnn-2d-basic-solution-powered-by-fast-ai

# We load the requiere libraries
import numpy as np
import pandas as pd 
import os
# import git
import librosa
import matplotlib.pyplot as plt
import gc

from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import label_ranking_average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import zipfile

tqdm.pandas()


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

In [0]:
!unzip "/content/drive/My Drive/Augmented_Features.zip"

Archive:  /content/drive/My Drive/Augmented_Features.zip
   creating: Augmented_Features/
  inflating: Augmented_Features/albanian1.pkl  
  inflating: Augmented_Features/albanian10.pkl  
  inflating: Augmented_Features/albanian11.pkl  
  inflating: Augmented_Features/albanian12.pkl  
  inflating: Augmented_Features/albanian13.pkl  
  inflating: Augmented_Features/albanian14.pkl  
  inflating: Augmented_Features/albanian15.pkl  
  inflating: Augmented_Features/albanian16.pkl  
  inflating: Augmented_Features/albanian17.pkl  
  inflating: Augmented_Features/albanian18.pkl  
  inflating: Augmented_Features/albanian19.pkl  
  inflating: Augmented_Features/albanian2.pkl  
  inflating: Augmented_Features/albanian20.pkl  
  inflating: Augmented_Features/albanian21.pkl  
  inflating: Augmented_Features/albanian22.pkl  
  inflating: Augmented_Features/albanian23.pkl  
  inflating: Augmented_Features/albanian24.pkl  
  inflating: Augmented_Features/albanian25.pkl  
  inflating: Augmented_Feature

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [0]:
import pandas as pd       
import os 
import math 
import numpy as np
import matplotlib.pyplot as plt  
import IPython.display as ipd  # To play sound in the notebook
import librosa
import librosa.display
import time
import warnings
import pickle as pickle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
SAMPLE_RATE = 22050
warnings.filterwarnings('ignore')
df = pd.read_csv("sample_data/speakers_all.csv")
# df = pd.read_csv("drive/My Drive/aug_speakers.csv")
df.drop(df.columns[9:12],axis = 1, inplace = True)
print(df.columns)
df.describe()

Index(['age', 'age_onset', 'birthplace', 'filename', 'native_language', 'sex',
       'speakerid', 'country', 'file_missing?'],
      dtype='object')


,age,age_onset,speakerid
count,3621.000000,3621.000000,3621.000000
mean,32.852251,10.044877,1073.967688
std,14.022968,8.579382,628.435246
min,0.000000,0.000000,1.000000
25%,22.000000,5.000000,538.000000
50%,28.000000,10.000000,1048.000000
75%,41.000000,13.000000,1616.000000
max,97.000000,86.000000,2176.000000


In [0]:
# import collections
# samples_count = collections.Counter(df["native_language"].values)
# new_d = {}
# for key,val in samples_count.items():
#     new_d[val] = new_d.get(val,[])+[key]

# for i in range(8):
#     if i in new_d.keys():
#         del new_d[i]

# # print(new_d)
# sorted_d = {k: v for k, v in sorted(new_d.items(), key=lambda item: item[0])}
# classes = []
# for _,val in sorted_d.items():
#     classes+=val
# sorted_d.clear()
# new_d.clear()

# print(total)
# print(len(classes))
# {k: v for k, v in sorted(new_d.items(), key=lambda item: item[0])}

In [0]:
# def freq_based_function(f_miss, fname):
#     if not f_miss:
#         try:
#             path = 'sample_data/recordings/' + fname +".mp3"
#             data, sampling_rate = librosa.load(path, sr=SAMPLE_RATE)
#             if 0 < len(data): # workaround: 0 length causes error
#                 y, _ = librosa.effects.trim(data) # trim, top_db=default(60)
#             # make it unified length to conf.samples
#             trim_long_data = True
#             if len(y) > 22050*15: # long enough
#                 if trim_long_data:
#                     y = y[0:0+22050*15]
#             else: # pad blank
#                 padding = 22050*15 - len(y)    # add padding at both ends
#                 offset = padding // 2
#                 y = np.pad(y, (offset, 22050*15 - len(y) - offset), 'constant')
                
#             duration = librosa.get_duration(y=y, sr=sampling_rate)
#             mfcc = librosa.feature.mfcc(y=y, sr=sampling_rate, n_mfcc = 40)
#             melspec = librosa.feature.melspectrogram(y, sr=sampling_rate, n_mels=128, fmin = 20,fmax = 22050 // 2,
#     n_fft = 128 * 20)
#             save_data = {"data":data,"sampling_rate":sampling_rate,"duration":duration
#                        ,"mfcc": mfcc,"melspec":melspec}
#             with open('sample_data/features/'+fname+'.pkl', 'wb') as outfile:
#                 pickle.dump(save_data, outfile, pickle.HIGHEST_PROTOCOL)
#         except Exception:
#             return 

# # durations,mfccs,melspecs,melspec_hs,melspec_ps,y_harmonics,y_percussives,log_hs,log_ps,datas,sampling_rates = [],[],[],[],[],[],[],[],[],[],[]
# for _, row in df.iterrows():
#     print(_)
#     params = row[['file_missing?', 'filename']].values
#     freq_based_function(params[0],params[1])


In [0]:
start_time = time.time()

def extract_features_from_pkl(fname):
    try:
        # path = '/content/drive/My Drive/aug_features/' + fname +".pkl"
        path = 'Augmented_Features/' + fname +".pkl"
        extracted_data = pd.read_pickle(path)
        # data, sampling_rate = extracted_data["data"], extracted_data["sampling_rate"]
        mfcc = extracted_data["mfcc"]
        melspec = extracted_data["melspec"]
        return (mfcc,melspec)
    except Exception:
        return 
mfccs,melspecs = [],[]
for _, row in df.iterrows():
    print(_)
    val = extract_features_from_pkl(row['filename'])
    if val:
        mfccs.append(val[0])
        melspecs.append(val[1])
        # datas.append(val[2])
        # sampling_rates.append(val[3])
    else:
        mfccs.append(None)
        melspecs.append(None)
        # datas.append(None)
        # sampling_rates.append(None)
        

print("time taken "+str((time.time() - start_time)/60))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
mfccs

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 array([[-573.69913689, -573.58495864, -573.69657426, ..., -322.28808653,
         -386.00228225, -399.08549092],
        [   3.76159089,    3.92294105,    3.76520739, ...,  174.34897297,
          155.9134371 ,  147.44190241],
        [   3.76074115,    3.92172461,    3.76433481, ...,   53.01678135,
           74.14214541,   76.28302683],
        ...,
        [   3.38070025,    3.39936599,    3.37552691, ...,   -3.0194506 ,
           -1.19188812,    0.84456888],
        [   3.36019741,    3.37237004,    3.3546301 , ...,   -6.18976359,
            1.69267933,    3.37661688],
        [   3.33918855,    3.34482857,    3.33322558, ...,   -5.31338293,
           -3.44833318,    1.25082169]]),
 array([[-638.60910719, -638

In [0]:
df["mfcc"] = mfccs
df["melspec"] = melspecs
df = df.sample(frac=1).reset_index(drop=True)
# df["librosa_data"] = datas
# df["librosa_sampling_rate"] = sampling_rates

In [0]:
# mfccs = None
# melspecs = None

In [0]:
df.tail()

,age,age_onset,birthplace,filename,native_language,sex,speakerid,country,file_missing?,mfcc,melspec
3616,26.0,10.0,"hoshiarpur, india",punjabi43,punjabi,male,292,india,False,"[[-446.88931706122304, -448.3817458634143, -46...","[[0.007157872661286311, 0.010988244742904269, ..."
3617,52.0,0.0,"oxford, uk",english365,english,male,1292,uk,False,None,None
3618,23.0,5.0,"islamabad, pakistan",urdu30,urdu,male,1061,pakistan,False,"[[-542.5207570756938, -547.0848026662861, -549...","[[0.000768369329629753, 0.0004267779691255449,..."
3619,27.0,12.0,"tokyo, japan",japanese2,japanese,female,222,japan,False,"[[-395.63500792915323, -388.92534640331826, -3...","[[0.0020306017583537495, 0.0013261725325752673..."
3620,21.0,11.0,"lamentin, martinique",french9,french,male,412,martinique,False,"[[-239.2346056057646, -235.28785519325058, -24...","[[0.03374833616115063, 0.018363277807351842, 0..."


In [0]:
df = df.dropna()
df.drop(columns=['age_onset','filename','speakerid','file_missing?','birthplace','country'],inplace=True)
# df.shape

In [0]:
labels = np.sort(np.unique(df['native_language'].values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c


def test(index):
    global num_class
    labels_array = np.zeros((num_class))
    labels_array[index] = 1
    return labels_array

one_hot_labels = [test(c2i[x]) for x in df['native_language'].values]
df["native_lang_c2i"] = one_hot_labels
df["native_lang_c2i"].head()
for f in labels:
    df[f] = 0.0
    #train_noisy[f] = 0.0

df[labels] = one_hot_labels
df.tail()

one_hot_labels = None
labels = None

# def split_and_label(rows_labels):
    
#     row_labels_list = []
#     for row in rows_labels:
#         row_labels = row.split(',')
#         labels_array = np.zeros((80))
        
#         for label in row_labels:
#             index = label_mapping[label]
#             labels_array[index] = 1
        
#         row_labels_list.append(labels_array)
    
#     return row_labels_list


# train_curated_labels = split_and_label(df['labels'])

In [0]:
# def get_mfcc(df):
#     X = []
#     for i, row in df.iterrows():
#         x = row["mfcc"]
#         X.append(x.transpose())
#     return X

# X = np.array(get_mfcc(df))
def get_melspec(df):
    X = []
    for i, row in df.iterrows():
        x = row["melspec"]
        spectrogram = librosa.power_to_db(x)
        spectrogram = spectrogram.astype(np.float32)
        X.append(x.transpose())
    return X

X = np.array(get_melspec(df))

In [0]:
Y = np.array(df.iloc[:,-51:])

In [0]:
# df.drop(df.index, inplace=True)

In [0]:
X.shape

(2550, 776, 128)

In [0]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [0]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [0]:
# This allows to the transformer to know where there is real data and where it is padded
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [0]:
def encoder_layer(units, d_model, num_heads, dropout,name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model ), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def encoder(time_steps,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            projection,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
  
  if projection=='linear':
    ## We implement a linear projection based on Very Deep Self-Attention Networks for End-to-End Speech Recognition. Retrieved from https://arxiv.org/abs/1904.13377
    projection=tf.keras.layers.Dense( d_model,use_bias=True, activation='linear')(inputs)
    print('linear')
  
  else:
    projection=tf.identity(inputs)
    print('none')
   
  projection *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  projection = PositionalEncoding(time_steps, d_model)(projection)

  outputs = tf.keras.layers.Dropout(rate=dropout)(projection)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])
 
 
  

 
  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [0]:
def transformer(time_steps,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                output_size,
                projection,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  
  
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(tf.dtypes.cast(
          
    #Like our input has a dimension of length X d_model but the masking is applied to a vector
    # We get the sum for each row and result is a vector. So, if result is 0 it is because in that position was masked      
    tf.math.reduce_sum(
    inputs,
    axis=2,
    keepdims=False,
    name=None
), tf.int32))
  

  enc_outputs = encoder(
      time_steps=time_steps,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      projection=projection,
      name='encoder'
  )(inputs=[inputs, enc_padding_mask])

  #We reshape for feeding our FC in the next step
  outputs=tf.reshape(enc_outputs,(-1,time_steps*d_model))
  
  #We predict our class
  outputs = tf.keras.layers.Dense(units=output_size,use_bias=True,activation='softmax', name="outputs")(outputs)

  return tf.keras.Model(inputs=[inputs], outputs=outputs, name='audio_class')

In [0]:
#We get our train and test set
X_train,X_test, Y_train, Y_test =train_test_split(X,Y, test_size=0.2, random_state=27)
X_train,X_val, Y_train, Y_val = train_test_split(X_train,Y_train,test_size=0.25,random_state=27)



In [0]:
Y_train.shape

(1530, 51)

In [0]:
print(X_train.shape,X_test.shape)

(1530, 776, 40) (510, 776, 40)


In [0]:
# projection=['linear','none']
projection = ['linear']
accuracy=[]
proj_implemented=[]

In [0]:

NUM_LAYERS = 2
D_MODEL = X.shape[2]
NUM_HEADS = 4
UNITS = 5
DROPOUT = 0.1
TIME_STEPS= X.shape[1]
OUTPUT_SIZE=51
EPOCHS = 50
EXPERIMENTS=1

model = transformer(time_steps=TIME_STEPS,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    output_size=OUTPUT_SIZE,  
    projection="linear")
for j in range(EXPERIMENTS):
  # lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  # 0.0005,
  # decay_steps=0.9,
  # decay_rate=1,
  # staircase=False)
  model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


  history=model.fit(X_train, Y_train, epochs=EPOCHS, validation_data=(X_test, Y_test))

  accuracy.append(max(history.history['val_accuracy']))
    
  proj_implemented.append("linear")
    
    # del model
    
  del history
      
   


linear
Epoch 1/50
48/48 [==============================] - 13s 267ms/step - loss: 5.9410 - accuracy: 0.0458 - val_loss: 5.6917 - val_accuracy: 0.0471
Epoch 2/50
48/48 [==============================] - 12s 253ms/step - loss: 2.9311 - accuracy: 0.3020 - val_loss: 5.4583 - val_accuracy: 0.0608
Epoch 3/50
48/48 [==============================] - 12s 254ms/step - loss: 1.2877 - accuracy: 0.6523 - val_loss: 5.3550 - val_accuracy: 0.0667
Epoch 4/50
48/48 [==============================] - 12s 255ms/step - loss: 0.5745 - accuracy: 0.8542 - val_loss: 5.3399 - val_accuracy: 0.0608
Epoch 5/50
48/48 [==============================] - 12s 255ms/step - loss: 0.3017 - accuracy: 0.9307 - val_loss: 5.4208 - val_accuracy: 0.0667
Epoch 6/50
48/48 [==============================] - 12s 255ms/step - loss: 0.1668 - accuracy: 0.9712 - val_loss: 5.3411 - val_accuracy: 0.0549
Epoch 7/50
48/48 [==============================] - 12s 259ms/step - loss: 0.0873 - accuracy: 0.9882 - val_loss: 5.3035 - val_accuracy:

In [0]:
del model

In [0]:
accuracy=pd.DataFrame(accuracy, columns=['accuracy'])
proj_implemented=pd.DataFrame(proj_implemented, columns=['projection'])
results=pd.concat([accuracy,proj_implemented],axis=1)


In [0]:
y_preds = model.predict(X_val)

score = model.evaluate(X_val, Y_val,  verbose=1)

print("score",score)

In [0]:
results.groupby('projection').mean()


,accuracy
projection,
linear,0.076471


In [0]:
results.groupby('projection').std()

In [0]:
df["duration"].min()